In [1]:
from IPython import embed
from dotenv import load_dotenv
from jupyter_server.auth.security import persist_config

# from langchain_llm_test import ai_message
%pip install --upgrade --quiet  docx2txt langchain-community



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU langchain-text-splitters



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain-chroma


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -U langchain langchainhub --quiet



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax_docs/tax_with_table.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
len(document_list)

225

In [7]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-large"
)

In [8]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, persist_directory="./chroma", collection_name="chroma-tax")
database = Chroma(persist_directory="./chroma", collection_name="chroma-tax", embedding_function=embedding)

In [9]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# retrieved_docs = database.similarity_search(query, 3)

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')


In [11]:
# ai_message = llm.invoke(prompt)

In [12]:
# ai_message.content

In [13]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [15]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [16]:
ai_message = qa_chain({"query": query})

/var/folders/t1/msm3rkc1453dsn7x4j1bkh2h0000gn/T/ipykernel_77989/314547319.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [17]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만 원인 직장인의 소득세는 기본세율이 적용됩니다. 기본세율은 소득 구간에 따라 달라지며, 구체적인 세율과 공제 금액은 각 개인의 상황에 따라 다를 수 있어 정확한 소득세를 계산하려면 세율 표와 공제 내역을 참고해야 합니다. 일반적으로 근로소득에 대한 소득세는 누진세 방식으로 부과됩니다.'}